In [ ]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
from scipy import stats
import plotly.express as px
from scripts.python.routines.plot.scatter import add_scatter_trace
import plotly.graph_objects as go
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout, get_axis
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=False)
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import seaborn as sns
from glob import glob
import pathlib
from sklearn.metrics import mean_absolute_error
from scipy import stats
import patchworklib as pw
import os
import functools
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import shap
from slugify import slugify
from src.models.tabular.widedeep.ft_transformer import WDFTTransformerModel
from art.estimators.regression.pytorch import PyTorchRegressor
from art.estimators.regression.blackbox import BlackBoxRegressor
from art.attacks.evasion import LowProFool, ZooAttack, FastGradientMethod
import torch


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

In [ ]:
path = "D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"

path_save = f"{path}/special/048_yakutia_table_update"
pathlib.Path(f"{path}").mkdir(parents=True, exist_ok=True)

df_dnam = pd.read_excel(f"{path}/pheno.xlsx", index_col="index")
df_immuno = pd.read_excel(f"{path_save}/immuno_pred.xlsx", index_col="index")

df_yak_l = pd.read_excel(f"{path_save}/origin.xlsx", sheet_name="Долгожители", index_col=0)
df_yak_c = pd.read_excel(f"{path_save}/origin.xlsx", sheet_name="Контрольный", index_col=0)

In [ ]:
yak_l_subjs = df_yak_l.loc[df_yak_l['Код Epigen 2022'].notna(), 'Код Epigen 2022'].values
yak_l_index = df_yak_l.index[df_yak_l['Код Epigen 2022'].isin(yak_l_subjs)].values
diff_yak_l_dnam = list(set(yak_l_subjs) - set(df_dnam.index.values))
print(f"diff_yak_l_dnam: {diff_yak_l_dnam}")
diff_yak_l_immuno = list(set(yak_l_subjs) - set(df_immuno.index.values))
print(f"diff_yak_l_immuno: {diff_yak_l_immuno}")

yak_c_subjs = df_yak_c.loc[df_yak_c['Код Epigen 2022'].notna(), 'Код Epigen 2022'].values
yak_c_index = df_yak_c.index[df_yak_c['Код Epigen 2022'].isin(yak_c_subjs)].values
diff_yak_c_dnam = list(set(yak_c_subjs) - set(df_dnam.index.values))
print(f"diff_yak_c_dnam: {diff_yak_c_dnam}")
diff_yak_c_immuno = list(set(yak_c_subjs) - set(df_immuno.index.values))
print(f"diff_yak_c_immuno: {diff_yak_c_immuno}")

dnam_suffix = "_harm"
age_types = {
    f"DNAmAgeHannum{dnam_suffix}": "Hannum",
    f"DNAmAge{dnam_suffix}": "Horvath",
    f"DNAmPhenoAge{dnam_suffix}": "PhenoAge",
    f"DNAmGrimAge{dnam_suffix}": "GrimAge",
}
for x in ["PCHorvath1", "PCHorvath2", "PCHannum", "PCPhenoAge", "PCGrimAge"]:
    age_types[x] = x

for yak_l_index in yak_l_index:
    subj = df_yak_l.at[yak_l_index, 'Код Epigen 2022']
    for at in age_types:
        df_yak_l.at[yak_l_index, age_types[at]] = df_dnam.at[subj, at]

    df_yak_l.at[yak_l_index, "SImAge"] = df_immuno.at[subj, 'Prediction']
df_yak_l.to_excel(f"{path_save}/df_yak_l.xlsx")

for yak_c_index in yak_c_index:
    subj = df_yak_c.at[yak_c_index, 'Код Epigen 2022']
    for at in age_types:
        df_yak_c.at[yak_c_index, age_types[at]] = df_dnam.at[subj, at]
    df_yak_c.at[yak_c_index, "SImAge"] = df_immuno.at[subj, 'Prediction']
df_yak_c.to_excel(f"{path_save}/df_yak_c.xlsx")